In [17]:
import numpy as np
import tensorflow as tf
from music21 import *
import os
import time
from tqdm.notebook import tqdm

In [18]:
path = os.getcwd()
filenames = os.listdir("./data")

In [19]:
def _parseMidi(list_filenames):
    print("Loading samples.....")
    samples = [converter.parse(path + "/data/" + str(file)) for file in tqdm(list_filenames)]
    sam_mono = [sample.chordify() for sample in tqdm(samples) if len(instrument.partitionByInstrument(sample).parts) == 1]
    print("Samples loaded and chordified.....")
    list_chords = []
    list_durations = []
    list_keys = []
    print("Preparing data....")
    for song in tqdm(out):
        list_keys.append(song.analyze("key"))
        for element in song:
            # These notes are usually not present after chordify() but still we don't want to miss any if present 
            if isinstance(element, note.Note):
                list_chords.append(element.pitch)
                list_durations.append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                list_chords.append(".".join(str(n) for n in element.pitches))
                list_durations.append(element.duration.quarterLength)
    return list_chords, list_durations, list_keys

In [20]:
list_chords, list_durations, list_keys = _parseMidi(filenames)

In [50]:
list_chords = [[] for _ in out]
list_durations = [[] for _ in out]
list_keys = [[]]
for i, song in enumerate(out):
    list_keys.append(song.analyze("key"))
    for element in song:
        if isinstance(element, note.Note):
            list_chords[i].append(element.pitch)
            list_durations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            list_chords[i].append(".".join(str(n) for n in element.pitches))
            list_durations[i].append(element.duration.quarterLength)

In [58]:
#Find the number of unique Chords
unique_chords = np.unique([i for s in list_chords for i in s])
print(len(unique_chords))
chord_to_int = dict(zip(unique_chords, range(0, len(unique_chords))))
#Find the number of unique durations
unique_durations = np.unique([i for s in list_durations for i in s])
print(len(unique_durations))
duration_to_int = dict(zip(unique_durations, range(0, len(unique_durations))))

12632
25


In [67]:
#Create the reverse dict
int_to_chord = {i:j for j, i in chord_to_int.items()}
int_to_durations = {i:j for j, i in duration_to_int.items()}

In [88]:
# Define the sequence length
sequence_length = 32

# Define the empty lists
train_chords = []
train_durations = []
target_chords = []
target_durations = []

#Genrate the train and the target data
for i in range(len(list_chords)):
    chordList = [chord_to_int[chord] for chord in list_chords[i]]
    durationList = [duration_to_int[chord] for chord in list_durations[i]]
    for j in range(len(chordList) - sequence_length - 1):
        train_chords.append(chordList[j:j + sequence_length])
        train_durations.append(durationList[j:j + sequence_length])
        target_chords.append(chordList[j + sequence_length + 1])
        target_durations.append(durationList[j + sequence_length + 1])

In [90]:
train_chords = np.asarray(train_chords)
train_durations = np.asarray(train_durations)
target_chords = np.asarray(target_chords)
target_durations = np.asarray(target_durations)

In [95]:
n_samples = train_chords.shape[0]
n_chords = train_chords.shape[1]
n_durations = train_durations.shape[1]

embed_dim = 64

In [162]:
# Input layers
chord_input = tf.keras.layers.Input((None,))
duration_input = tf.keras.layers.Input((None,))

# Embedding layers
embedding_chord = tf.keras.layers.Embedding(input_dim=len(chord_to_int), output_dim=embed_dim,
                                            input_length=sequence_length)(chord_input)
embedding_durations = tf.keras.layers.Embedding(input_dim=len(duration_to_int), output_dim=embed_dim, 
                                               input_length=sequence_length)(duration_input)

# Concat these 2 layers
concat = tf.keras.layers.Concatenate(axis = 1)([embedding_chord, embedding_durations])

# Define the single LSTM layer with 512 units
lstm_layer = tf.keras.layers.LSTM(512, return_sequences=True)(concat)

# Define the intermediate dense layer
dense = tf.keras.layers.Dense(256)(lstm_layer)

# Define the final output layers
dense1 = tf.keras.layers.Dense(len(chord_to_int), activation = "softmax")(dense)
dense2 = tf.keras.layers.Dense(len(duration_to_int), activation = "softmax")(dense)

# Define the model 
model = tf.keras.models.Model([chord_input, duration_input], [dense1, dense2])

In [163]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, None, 64)     808448      input_20[0][0]                   
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, None, 64)     1600        input_21[0][0]                   
____________________________________________________________________________________________

In [164]:
model.compile(loss = "sparse_categorical_crossentropy", metrics = ["accuracy"], optimizer = "rmsprop")

In [165]:
model.fit([train_chords, train_durations], [target_chords.reshape(-1,1), target_durations.reshape(-1,1)], batch_size=64, epochs = 10)

Train on 59365 samples
Epoch 1/10
   64/59365 [..............................] - ETA: 1:28:26

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  assertion failed: [Condition x == y did not hold element-wise:] [x (loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [64 1] [y (loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [64 64]
	 [[node loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at <ipython-input-165-1a2a6f682666>:1) ]]
	 [[Reshape_35/_92]]
  (1) Invalid argument:  assertion failed: [Condition x == y did not hold element-wise:] [x (loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [64 1] [y (loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [64 64]
	 [[node loss/dense_23_loss/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at <ipython-input-165-1a2a6f682666>:1) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_46739]

Function call stack:
distributed_f